#HR Analytics Dashboard: From Raw Data to Business Insights

##Overview
An end-to-end ETL pipeline that transforms messy employee data into actionable business intelligence.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

##DATA EXTRACTION ,DATA CLEANING & TRANSFORMATION
Load raw employee dataset,Handle multiple file formats,Remove duplicates & handle missing values
and Feature engineering (tenure, salary levels)


In [2]:
#extract data
def extract(file_name):
  return pd.read_csv(file_name)
#transform data
def transform(data_frame):
  data_frame = data_frame.replace("N/A", np.nan)
  data_frame=data_frame.drop_duplicates()
  data_frame['Age'] = data_frame.groupby('Department_Region')['Age'].transform(lambda x: x.fillna(np.round(x.mean())).astype(int))
  # Split Department_Region into two columns
  data_frame[['Department', 'Region']] = data_frame['Department_Region'].str.split('-', expand=True)

  # Fix dates
  data_frame['Join_Date'] = pd.to_datetime(data_frame['Join_Date'], errors='coerce')

  #  Calculate tenure (years at company)
  data_frame['Tenure'] = np.round((pd.to_datetime('today') - data_frame['Join_Date']).dt.days / 365.25,1)

  # Create salary categories
  data_frame['Salary_Level'] = pd.cut(data_frame['Salary'],
                                    bins=[0, 50000, 80000, 120000, float('inf')],
                                    labels=['Junior', 'Mid', 'Senior', 'Executive'])
  data_frame=data_frame.dropna()
  data_frame.loc[data_frame['Phone'] < 0, 'Phone'] = data_frame.loc[data_frame['Phone'] < 0, 'Phone'].abs()
  return data_frame
#load data
def load(data_frame,target):
  return data_frame.to_csv(target)




In [3]:
data=extract("Messy_Employee_dataset.csv")
data.head()
transformed_data=transform(data)
load(transformed_data,"Cleaned_Employees_dataset.csv")

##Bussiness Intelligence and Data Visualisation


In [4]:
def business_insight(data_frame):
    print("\nBUSINESS INSIGHTS")
    insights = {}
    #Annual Hiring Trend
    hiring_trend = data_frame.groupby(data_frame['Join_Date'].dt.year).size()
    insights['annual_hiring_trend'] = hiring_trend
    #EMPLOYEES age ratio across departments
    age_ratio = data_frame.groupby('Department')['Age'].mean()
    insights['age_ratio'] = age_ratio
    # Department Performance Analysis
    dept_stats = data_frame.groupby('Department').agg({
        'Salary': 'mean',
        'Performance_Score': lambda x: (x == 'Exceeds').mean(),
        'Employee_ID': 'count'
    }).round(2)

    dept_stats.columns = ['Avg_Salary', 'Top_Performer_Rate', 'Employee_Count']
    insights['department_analysis'] = dept_stats

    # Regional Distribution
    region_counts = data_frame['Region'].value_counts()
    insights['regional_distribution'] = region_counts

    # Salary Analysis
    salary_stats = {
        'total_payroll': f"${data_frame['Salary'].sum():,}",
        'average_salary': f"${data_frame['Salary'].mean():,.0f}",
        'highest_paid_dept': data_frame.groupby('Department')['Salary'].mean().idxmax(),
        'remote_vs_office_salary': data_frame.groupby('Remote_Work')['Salary'].mean().round(2)
    }
    insights['salary_analysis'] = salary_stats
    retention_risk = data_frame.groupby('Department').agg({
        'Tenure': 'mean',
        'Performance_Score': lambda x: (x == 'Below').mean(),
        'Salary': lambda x: (x < data_frame['Salary'].quantile(0.5)).mean()
    }).round(3)

    retention_risk.columns = ['Avg_Tenure', 'Low_Performer_Rate', 'Below_Median_Salary_Rate']

    # Calculate retention risk score (higher score = higher risk)
    retention_risk['Retention_Risk_Score'] = (
        retention_risk['Low_Performer_Rate'] * 0.4 +
        retention_risk['Below_Median_Salary_Rate'] * 0.3 +
        (1 - (retention_risk['Avg_Tenure'] / retention_risk['Avg_Tenure'].max())) * 0.3
    ).round(3)

    insights['retention_risk_analysis'] = retention_risk.sort_values('Retention_Risk_Score', ascending=False)


    return insights

def dashboard_visualizations(df, insights, out_file='HR Analytics Dashboard.png'):
    print("\nCreating visualizations...")

    # create subplots
    fig, axes = plt.subplots(3,2, figsize=(15, 12))

    # Annual Hiring Trend
    ax = axes[0, 0]
    if 'annual_hiring_trend' in insights and not insights['annual_hiring_trend'].empty:
        insights['annual_hiring_trend'].plot(kind='bar', ax=ax)
        ax.set_title('Annual Hiring Trend', fontweight='bold')
        ax.set_xlabel('Year')
        ax.set_ylabel('New Hires')
    else:
        ax.text(0.5, 0.5, 'No join date data', ha='center')
        ax.set_axis_off()

    # Average Salary by Department
    ax = axes[0, 1]
    if 'department_analysis' in insights and not insights['department_analysis'].empty:
        insights['department_analysis']['Avg_Salary'].sort_values().plot(kind='barh', ax=ax)
        ax.set_title('Average Salary by Department', fontweight='bold')
        ax.set_xlabel('Salary ($)')
    else:
        ax.text(0.5, 0.5, 'No department/salary data', ha='center')
        ax.set_axis_off()

    # 3) Employee Distribution by Region
    ax = axes[1, 0]
    if 'regional_distribution' in insights and not insights['regional_distribution'].empty:
        insights['regional_distribution'].plot(kind='pie', ax=ax, autopct='%1.1f%%', startangle=90)
        ax.set_ylabel('')
        ax.set_title('Employee Distribution by Region', fontweight='bold')
    else:
        ax.text(0.5, 0.5, 'No region data', ha='center')
        ax.set_axis_off()

    # Performance: Remote vs Office
    ax = axes[1, 1]
    if 'Remote_Work' in df.columns and 'Performance_Score' in df.columns:
        remote_perf = df.groupby(['Remote_Work', 'Performance_Score']).size().unstack(fill_value=0)
        if not remote_perf.empty:
            remote_perf.plot(kind='bar', stacked=True, ax=ax)
            ax.set_title('Performance: Remote vs Office', fontweight='bold')
            ax.set_ylabel('Number of Employees')
        else:
            ax.text(0.5, 0.5, 'No performance/remote data', ha='center')
            ax.set_axis_off()
    else:
        ax.text(0.5, 0.5, 'No performance/remote data', ha='center')
        ax.set_axis_off()
    # Average age of employees by Department
    ax = axes[2, 0]
    if 'age_ratio' in insights and not insights['age_ratio'].empty:
        insights['age_ratio'].sort_values().plot(kind='pie', ax=ax, autopct=lambda p: f'{p/100 * sum(insights["age_ratio"]):.1f}', startangle=90)
        ax.set_title('Average age of employees by Department', fontweight='bold')
        ax.set_ylabel('')
    else:
        ax.text(0.5, 0.5, 'No department/age data', ha='center')
        ax.set_axis_off()
    #Department Retention Risk Analysis
    ax = axes[2, 1]
    if 'retention_risk_analysis' in insights and not insights['retention_risk_analysis'].empty:
        risk_data = insights['retention_risk_analysis']['Retention_Risk_Score']
        colors = ['red' if x > risk_data.median() else 'orange' if x > risk_data.quantile(0.25) else 'green' for x in risk_data]

        risk_data.sort_values().plot(kind='barh', ax=ax, color=colors)
        ax.set_title('Department Retention Risk Score', fontweight='bold')
        ax.set_xlabel('Risk Score (Higher = More Risk)')
        for i, (dept, score) in enumerate(risk_data.sort_values().items()):
            risk_level = "High" if score > risk_data.median() else "Medium" if score > risk_data.quantile(0.25) else "Low"
            ax.text(score + 0.01, i, risk_level, va='center', fontsize=9)
    else:
        ax.text(0.5, 0.5, 'No retention risk data', ha='center')
        ax.set_axis_off()

    plt.tight_layout()
    plt.savefig(out_file, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Dashboard saved as '{out_file}'")


##Professional executive summary report

In [5]:
def executive_summary(df, out_file='Executive_Summary.txt'):
    # Calculate key metrics
    total_employees = len(df)
    total_payroll = df['Salary'].sum()
    avg_salary = df['Salary'].mean()
    remote_count = len(df[df['Remote_Work'] == True])
    office_count = len(df[df['Remote_Work'] == False])

    # Department analysis
    dept_summary = df.groupby('Department').agg({
        'Salary': ['count', 'mean'],
        'Performance_Score': lambda x: (x == 'Exceeds').mean() if 'Exceeds' in x.values else 0
    }).round(2)

    # Performance analysis
    performance_counts = df['Performance_Score'].value_counts()

    # Generate summary
    summary = f"""
EMPLOYEE ANALYTICS EXECUTIVE SUMMARY
Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}
===================================================================================================

METRICS:
• Total Employees: {total_employees:,}
• Total Annual Payroll: ${total_payroll:,.0f}
• Average Salary: ${avg_salary:,.0f}
• Remote Workforce: {remote_count} employees ({remote_count/total_employees*100:.1f}%)
===================================================================================================
PERFORMANCE BREAKDOWN:"""

    for score, count in performance_counts.items():
        summary += f"\n• {score}: {count} employees ({count/total_employees*100:.1f}%)"

    summary += f"\n\n DEPARTMENT OVERVIEW:"

    # Add top departments
    top_depts = df['Department'].value_counts().head(3)
    for dept, count in top_depts.items():
        dept_data = df[df['Department'] == dept]
        avg_sal = dept_data['Salary'].mean()
        summary += f"\n• {dept}: {count} employees, ${avg_sal:,.0f} avg salary"

    summary += f"""

 KEY INSIGHTS:
1. Largest department: {df['Department'].value_counts().index[0]} ({df['Department'].value_counts().iloc[0]} employees)
2. Highest paying department: {df.groupby('Department')['Salary'].mean().idxmax()} (${df.groupby('Department')['Salary'].mean().max():,.0f})
3. Remote work adoption: {remote_count/total_employees*100:.1f}% of workforce
4. Most common performance level: {performance_counts.index[0]}

"""

    # Save to file
    with open('Executive_Summary.txt', 'w') as f:
        f.write(summary)

    print(f"Executive summary saved as '{out_file}'")




In [6]:
insights = business_insight(transformed_data)
dashboard_visualizations(transformed_data,insights)
executive_summary(transformed_data)


BUSINESS INSIGHTS

Creating visualizations...
Dashboard saved as 'HR Analytics Dashboard.png'
Executive summary saved as 'Executive_Summary.txt'
